In [1]:
import numpy as np

In [6]:
a = np.array([[2,3,4,5],[1,2,3,4],[0,1,2,3]])
a.shape[0]
a.reshape(a.shape[0],a.shape[1],1)

array([[[2],
        [3],
        [4],
        [5]],

       [[1],
        [2],
        [3],
        [4]],

       [[0],
        [1],
        [2],
        [3]]])